# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
data[["CustomerID","ProductName","Quantity"]].head()

,CustomerID,ProductName,Quantity
0,61288,Bread - Hot Dog Buns,16
1,77352,Bread - Hot Dog Buns,20
2,40094,Bread - Hot Dog Buns,11
3,23548,Bread - Hot Dog Buns,6
4,78981,Bread - Hot Dog Buns,20


In [5]:
grouped = data[["CustomerID","ProductName","Quantity"]].groupby(by=["CustomerID","ProductName"]).sum().reset_index(drop=False)
test = grouped[grouped.CustomerID == 3317]
test


,CustomerID,ProductName,Quantity
2788,3317,Bay Leaf,1
2789,3317,Beef - Ground Medium,1
2790,3317,Beef - Rib Eye Aaa,1
2791,3317,Butter - Unsalted,1
2792,3317,Cheese - Victor Et Berthold,1
2793,3317,Corn Meal,1
2794,3317,"Crab - Dungeness, Whole",1
2795,3317,Crab - Imitation Flakes,1
2796,3317,Crush - Cream Soda,1
2797,3317,Cup - Translucent 7 Oz Clear,1


In [6]:
grouped.columns

Index(['CustomerID', 'ProductName', 'Quantity'], dtype='object')

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [7]:
pivoted = grouped.pivot_table(values="Quantity",index="ProductName",columns="CustomerID",fill_value=0)#.fillna(0)
pivoted.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [8]:
pivoted.columns

Int64Index([   33,   200,   264,   356,   412,   464,   477,   639,   649,
              669,
            ...
            97697, 97753, 97769, 97793, 97900, 97928, 98069, 98159, 98185,
            98200],
           dtype='int64', name='CustomerID', length=1000)

In [9]:
#distances = pdist(pivoted,"euclidean")
#print(squareform(distances))
distances2 = pd.DataFrame(1/(1 + squareform(pdist(pivoted.T, 'euclidean'))),
                         index=pivoted.columns, columns=pivoted.columns)
distances2.reset_index(level= 0)
distances2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [10]:
similar5 = distances2[33].sort_values(ascending=False)[1:6]
similar5

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [11]:
step5 = pd.DataFrame(columns= grouped.columns)
for item in similar5.keys():
    daf = grouped[grouped["CustomerID"] == item]
    step5 = pd.concat([daf,step5],ignore_index=True)
    #print(daf)
    
step5.head()

,CustomerID,ProductName,Quantity
0,3305,Bandage - Flexible Neon,1
1,3305,Beans - Wax,2
2,3305,Beef - Inside Round,1
3,3305,Beef - Rib Eye Aaa,1
4,3305,Beef Wellington,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [12]:
step6 = step5[["ProductName","Quantity"]].groupby(by=["ProductName"]).sum().sort_values("Quantity",ascending=False)
step6.head()

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [13]:
step7 = pivoted[pivoted[33]==0][33]


step7 = pd.concat([step7,step6], axis=1).fillna(0).sort_values(by="Quantity", ascending=False)
step7.head(5)

,33,Quantity
Butter - Unsalted,0.0,3.0
"Pepper - Black, Whole",0.0,3.0
Wine - Blue Nun Qualitatswein,0.0,3.0
Wine - Ej Gallo Sierra Valley,0.0,3.0
Towels - Paper / Kraft,0.0,3.0


In [14]:
step7.head(5).T

,Butter - Unsalted,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,Wine - Ej Gallo Sierra Valley,Towels - Paper / Kraft
33,0.0,0.0,0.0,0.0,0.0
Quantity,3.0,3.0,3.0,3.0,3.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [18]:
def to_dictionary(distancesdf):
    costumers = pivoted.columns
    recomendations = {}
    for item in costumers:
        similar_ones = distancesdf[item].sort_values(ascending=False)[1:6]
        data2 = pd.DataFrame()
        for similar in similar_ones.keys():
            df = grouped[grouped.CustomerID == similar]
            data2 = pd.concat([df,data2],ignore_index=True)
        grouped2 = data2[["ProductName","Quantity"]].groupby(by=["ProductName"]).sum().sort_values("Quantity",ascending=False)
        prov = pivoted[pivoted[item]==0][item]
        final = pd.concat([prov,grouped2], axis=1).fillna(0).sort_values(by="Quantity", ascending=False)
        recomendations.update({item: final.head(5).T.columns})
    return recomendations
    
    
recomendations2 = to_dictionary(distances2)    
    
    

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [20]:
pd.DataFrame.from_dict(recomendations2,orient='index').head()

,0,1,2,3,4
33,Butter - Unsalted,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,Wine - Ej Gallo Sierra Valley,Towels - Paper / Kraft
200,Soup - Campbells Bean Medley,Muffin - Carrot Individual Wrap,Lamb - Ground,Pork - Kidney,Bay Leaf
264,Bread - Italian Roll With Herbs,Lettuce - Spring Mix,"Veal - Inside, Choice",Fondant - Icing,Wine - Two Oceans Cabernet
356,"Veal - Inside, Choice",Sardines,Potatoes - Idaho 100 Count,"Tart Shells - Sweet, 4",Butter - Unsalted
412,"Veal - Inside, Choice",Olive - Spread Tapenade,Sprouts - Baby Pea Tendrils,"Sauce - Gravy, Au Jus, Mix",Cheese - Cambozola


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [25]:
distances3 = pd.DataFrame(1/(1 + squareform(pdist(pivoted.T, 'jaccard'))),
                         index=pivoted.columns, columns=pivoted.columns)
distances3.reset_index(level= 0)
distances3.head()

new_recomentations= to_dictionary(distances3)

In [26]:
pd.DataFrame.from_dict(new_recomentations,orient='index').head()

,0,1,2,3,4
33,Loquat,Wine - Crozes Hermitage E.,Wine - Wyndham Estate Bin 777,Blueberries,Scampi Tail
200,Muffin - Carrot Individual Wrap,Pomello,"Beans - Kidney, Canned",Table Cloth 81x81 White,Beef - Texas Style Burger
264,Sauce - Demi Glace,Flavouring - Orange,Wine - Two Oceans Cabernet,Lettuce - Spring Mix,Grouper - Fresh
356,"Veal - Inside, Choice",Potatoes - Idaho 100 Count,Corn Meal,Lettuce - Spring Mix,"Pepper - Black, Whole"
412,Wine - Wyndham Estate Bin 777,Cumin - Whole,"Tart Shells - Sweet, 4",Milk - 2%,Curry Paste - Madras
